In [22]:
from langchain_openai import ChatOpenAI

import os 
from dotenv import load_dotenv
load_dotenv()

api_key= os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI()

llm = ChatOpenAI(openai_api_key=api_key, max_tokens=100, temperature=0.7)

llm.invoke("what is a supernova?")

AIMessage(content='A supernova is a powerful and extremely energetic celestial event that occurs when a massive star reaches the end of its life cycle. It is a catastrophic explosion that results in the sudden increase in brightness of the star, temporarily outshining an entire galaxy before fading away over time.\n\nDuring a supernova, the star releases an enormous amount of energy and radiation, which causes the outer layers of the star to be expelled into space at an incredible velocity. This explosion can be so intense that for a short time')

In [23]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", ""),
    ("user", "{input}")
])

chain = prompt | llm 

chain.invoke({"input": ""})


AIMessage(content="As an AI language model, I don't have personal opinions or emotions. However, I can provide you with some common issues that people may dislike about Poland, but please note that these are generalizations and may not apply to everyone:\n\n1. Political climate: Some people may dislike the current political situation or specific policies in Poland.\n2. Corruption: Like in any country, corruption can be a concern for some individuals.\n3. Discrimination: Some people may dislike instances of discrimination based on race,")

In [24]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "what's the polish name of Jawad?"})

'The Polish name for Jawad is usually spelled as "Dżawad."'

In [25]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import HNLoader
loader = HNLoader("https://news.ycombinator.com/item?id=34817881")
data = loader.load()
# print(data[0].page_content[:300])
# data[0].metadata
# docs = data.load()
# {'source': 'https://news.ycombinator.com/item?id=34817881',
#  'title': 'What Lights the Universe’s Standard Candles?'}

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [27]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)
vector = FAISS.from_documents(documents, embeddings)

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "what is a supernova?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...



Based on the provided context, a supernova is an explosion that occurs in astrophysics when a star reaches the end of its life cycle.


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})


In [ ]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("example_data/layout-parser-paper.pdf")
pages = loader.load_and_split()